In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot as plt

In [2]:
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/sigma*sqrt(T)

def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [3]:
S, K, T, r, sig = 100,100,1,0.01,0.5
bs_call(S, K, T, r, sig)

20.144406289860115

In [4]:


def FD_BSM(S, K, T, r, sig, ns, nt, factor, type='European'):
    # B * F_{i-1} = C * F_{i}
    # F_{i-1} = B \ C * F_{i}
    S_max = K * factor
    ds = S_max / ns
    dt = T / nt

    # Lower bound: Dirichlet boundary. 0 spot price means 0 call option price
    # Upper bound: Zero Gamma boundary.

    V = np.zeros([ns,nt]) # value at each time step. t=i*dt, Sj = j*ds
    B = np.zeros([ns-1,ns-1])
    C = np.zeros([ns-1,ns-1])

    # Apply Gamma free BC 
    B[ns-2,ns-2] = 1.0
    B[ns-2,ns-3] = -2.0
    B[ns-2,ns-4] = 1.0

    # Assign payoff at maturity
    for j in range(V.shape[0]): # each row in the last column
        Sj = (j+1)*ds
        V[j,nt-1] = np.max([Sj-K,0.0])
    #    print(V[j,nt-1])

    # Build up B and C
    # Explicit scheme: v(i+1)-v(i)=v(i+1,j-1)*2a + v(i+1,j)*2b + v(i+1,j+1)*2c
    for j in range(B.shape[0]-1):
        jj = j+1 # j value for computing grid values
        a = 0.5*dt*(r*jj-(sig*jj)**2)/2
        b = 0.5*dt*((sig*jj)**2+r)
        c = 0.5*-dt*(r*jj+(sig*jj)**2)/2
    #    print(a,b,c)
        if j > 0:
            B[j,j-1] = a
            C[j,j-1] = -a
        B[j,j] = 1+b
        B[j,j+1] = c
        C[j,j] = 1-b
        C[j,j+1] = -c

    # Each time step excluding the very last
    for i in reversed(range(nt-1)):
        rhs = np.dot(C,V[1:,i+1])
        rhs[ns-2] = 0.0 # zero Gamma boundary
        V[1:,i] = np.linalg.solve(B, rhs)
        if type == 'American':
            for j in range(1,ns):
                V[j,i] = np.max([V[j,i], ds*j-K])

    # Return result by interpolation
    k_array = np.array(range(1,ns+1))*ds
    return np.interp(K,k_array,V[:,0])

def FD_BSM_log(S, K, T, r, sig, ns, nt, factor, type='European'):
    # log-normal price process
    # B * F_{i-1} = C * F_{i}
    # F_{i-1} = B \ C * F_{i}
    s = np.log(S)
    k = np.log(K)
    s_max = np.log(K*factor)
    ds = s_max/ns
    dt = T/nt
    
    # Lower bound: Dirichlet boundary. 0 spot price means 0 call option price
    # Upper bound: Zero Gamma boundary.
    V = np.zeros([ns,nt]) # value at each time step. t=i*dt, Sj = j*ds
    B = np.zeros([ns-1,ns-1])
    C = np.zeros([ns-1,ns-1])
    
    # Apply Gamma free BC 
    B[ns-2,ns-2] = 1/ds**2 - 1/(2*ds) # s(j+1)
    B[ns-2,ns-3] = -2/ds**2 # s(j)
    B[ns-2,ns-4] = 1/ds**2 + 1/(2*ds) # s(j-1)
    B[ns-2,ns-2] = 1.0
    B[ns-2,ns-3] = -2.0
    B[ns-2,ns-4] = 1.0
    
    # Assign payoff at maturity
    for j in range(V.shape[0]): # each row in the last column
        sj = (j+1)*ds
        V[j,nt-1] = np.max([np.exp(sj)-K,0.0])

    # Build up B and C
    # Explicit scheme: v(i+1)-v(i)=v(i+1,j-1)*2a + v(i+1,j)*2b + v(i+1,j+1)*2c
    for j in range(B.shape[0]-1):
        jj = j+1 # j value for computing grid values
        a = 0.5*dt*((r-0.5*sig**2)/(2*ds)-(sig**2)/(2*ds**2))
        b = 0.5*dt*(r+(sig/ds)**2)
        c = 0.5*dt*(-(r-0.5*sig**2)/(2*ds)-(sig**2)/(2*ds**2))
    #    print(a,b,c)
        if j > 0:
            B[j,j-1] = a
            C[j,j-1] = -a
        B[j,j] = 1+b
        B[j,j+1] = c
        C[j,j] = 1-b
        C[j,j+1] = -c

    # Each time step excluding the very last
    # for log-price, zero Gamma needs to be modified
    for i in reversed(range(nt-1)):
        rhs = np.dot(C,V[1:,i+1])
        rhs[ns-2] = 0.0 # zero Gamma boundary
        V[1:,i] = np.linalg.solve(B, rhs)
        if type == 'American':
            for j in range(1,ns):
                V[j,i] = np.max([V[j,i], ds*j-K])

    # Return result by interpolation
    k_array = np.array(range(1,ns+1))*ds
    return np.interp(k,k_array,V[:,0])


In [5]:
ns = 512 # number of price grids
nt = 400 # number of time steps
factor = 3
sig_arr = np.linspace(0.1,0.5,10)
BS_analytic = np.zeros(len(sig_arr))
BS_FD = np.zeros(len(sig_arr))
BS_FD_log = np.zeros(len(sig_arr))

for i,sig in enumerate(sig_arr):
    BS_analytic[i] = bs_call(S, K, T, r, sig)
    BS_FD[i] = FD_BSM(S, K, T, r, sig, ns=ns, nt=nt, factor=factor)
    BS_FD_log[i] = FD_BSM_log(S, K, T, r, sig, ns=ns, nt=nt, factor=factor)
    
rel_error = (BS_FD - BS_analytic)/ BS_analytic * 100
print(rel_error)
rel_error_log = (BS_FD_log - BS_analytic)/ BS_analytic * 100
print(rel_error_log)

[-0.69445455 -0.70450259 -0.70786045 -0.70912335 -0.70955011 -0.70958278
 -0.70940344 -0.70909747 -0.70873182 -0.70850395]
[ 0.00253843 -0.05978819 -0.08499754 -0.09757269 -0.10521343 -0.11536714
 -0.14627357 -0.23223687 -0.41854805 -0.75119242]


# American Option Pricing

In [6]:
# Binomial Tree for America and European options
# Author: Mehdi Bounouar,
# 
# The code is inspired from:
# [S.R Das, B. Ganger, Journal of Investment Management, Vol.9, No.4, (2010), pp.73-84]
# and the C++ code version from Volopta written by Dr. F. Rouah

def BinomialTreeCRR(n, Spot, k, r, v, T, PutCall, OpStyle):
    """
    n: steps
    Spot: Spot price
    K: Strike price
    r: Risk free rate
    v: volatility
    T: Maturity
    PutCall: European or American
    OpStyle: Call or Put
    """
    dt = T/n
    u = np.exp(v*np.sqrt(dt))
    d = 1./u
    p = (np.exp(r*dt)-d)/(u-d)
    
    #Binomial price tree
    stkval = np.zeros((n+1,n+1))
    stkval[0,0] = Spot
    for i in range(1,n+1):
        stkval[i,0] = stkval[i-1,0]*u
        for j in range(1,i+1):
            stkval[i,j] = stkval[i-1,j-1]*d
    
    #option value at each final node
    optval = np.zeros((n+1,n+1))
    for j in range(n+1):
        if PutCall=="C": # Call
            optval[n,j] = max(0, stkval[n,j]-k)
        elif PutCall=="P": #Put
            optval[n,j] = max(0, k-stkval[n,j])
    
    #backward recursion for option price
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            if OpStyle=="E":
                optval[i,j] = np.exp(-r*dt)*(p*optval[i+1,j] + (1-p)*optval[i+1,j+1])
            elif OpStyle=="A":
                if PutCall=="P":
                    optval[i,j] = max(k-stkval[i,j], np.exp(-r*dt)*(p*optval[i+1,j]+(1-p)*optval[i+1,j+1]))
                elif PutCall=="C":
                    optval[i,j] = max(stkval[i,j]-k, np.exp(-r*dt)*(p*optval[i+1,j]+(1-p)*optval[i+1,j+1]))
    return optval[0,0]

In [7]:
Spot = 100.           # Spot Price
k = 100.               # Strike Price
r = .01               # Annual Risk-free rate
v = sig             # Annual Volatility
T = 365./(365)        # Time in year (days/365)
n = 512                 # Number of steps
BinomialTreeCRR(n, Spot, k, r, v, T, PutCall="C", OpStyle="A")

20.13481805205403

In [8]:
FD_BSM(S, K, T, r, sig, ns=512, nt=nt, factor=factor,type='American')

20.00168237589211

In [9]:
FD_BSM(S, K, T, r, sig, ns=512, nt=nt, factor=factor,type='European')

20.00168237589211